<a href="https://colab.research.google.com/github/nisha1365/GEN-AI-POC/blob/main/Topic_Assignind_in_cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 1.5 MB/s eta 0:00:00


In [5]:
# Load the SentenceTransformer model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Load your CSV file into a pandas DataFrame
import pandas as pd
df = pd.read_csv('/content/topics_name_110.csv')

# Get the embeddings for all topics and cluster labels
import torch
topic_embeddings = model.encode(df['Topic_Name'].tolist())

cluster_labels = {
    'research': 'This is a research topic',
    'cost': 'This is a cost-related topic',
    'efficacy': 'This is an efficacy-related topic',
    'epidemiology': 'This is an epidemiology topic',
    'awareness': 'This is an awareness topic',
    'preference': 'This is a preference-related topic',
    'treatment': 'This is a treatment-related topic',
    'dummy1': 'This is a dummy topic'
}
label_embeddings = model.encode(list(cluster_labels.values()))

# Assign topics to cluster labels based on similarity
from scipy.spatial.distance import cosine
for i, topic_embedding in enumerate(topic_embeddings):
    similarities = [1 - cosine(topic_embedding, label_embedding) for label_embedding in label_embeddings]
    closest_label_index = similarities.index(max(similarities))
    closest_label = list(cluster_labels.keys())[list(cluster_labels.values()).index(list(cluster_labels.values())[closest_label_index])]
    df.at[i, 'Cluster_Label'] = closest_label

# Print the topics under each cluster label
for label in cluster_labels.keys():
    print(f"Cluster label: {label}")
    topics = df[df['Cluster_Label'] == label]['Topic_Name'].tolist()
    for topic in topics:
        print(f"\t{topic}")


Cluster label: research
Cluster label: cost
	Baby Care Logbook
	Financial Assistance for Baby's Health
	WEF Digital ID
	Pediatric Wellness Checkups
	Newsroom Exposé
Cluster label: efficacy
	Vaccine Safety for Infants
	Pfizer's Failure to Disclose Risks of Preterm Birth and Neonatal Death in Pregnant Women in RSV Vaccine Trial
	Immunization Schedule for Babies
	COVID-19 Vaccination Recommendations
	Vaccine Concerns
	COVID Vaccine Safety in Children
	COVID-19 Vaccination Recommendations
	Vaccine Safety in Children
	Vaccine Safety
	COVID-19 Vaccination Compensation
	Serum Run Nome
	Pediatric Nurse Qualifications
	Vaccine Safety in Children
	Vaccine Safety Research
	WHO guidance on clinical management of diphtheria
	Vaccine Safety Monitoring
	Vaccination Choice for Healthy Children
	Student Immunization Requirements
Cluster label: epidemiology
	Vaccination Exemption for School Children
	Vaccines for Tetanus and Chickenpox
	Hepatitis B Vaccination
	Vaccines and Infant Mortality
	Malaria Vac

Word Mover's Distance:
Word Mover's Distance (WMD): WMD is a measure of the distance between two text documents, which takes into account the semantic similarity between the words in the documents. You can use the wmdistance() function from the gensim library to calculate the WMD between each topic name and each cluster label, and then assign each topic to the cluster with the smallest WMD.

pip install POT

In [7]:
!pip install POT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 4.8 MB/s eta 0:00:00


In [16]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [23]:
# Load the SentenceTransformer and NLTK libraries
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import pandas as pd

# Load your CSV file into a pandas DataFrame
df = pd.read_csv('/content/topics_name_110.csv')

# Define the cluster labels and their descriptions
cluster_labels = {
    'research': 'This is a research topic',
    'cost': 'This is a cost-related topic',
    'efficacy': 'This is an efficacy-related topic',
    'epidemiology': 'This is an epidemiology topic',
    'awareness': 'This is an awareness topic',
    'preference': 'This is a preference-related topic',
    'treatment': 'This is a treatment-related topic',
    'dummy1': 'This is a dummy topic'
}

# Define a function to preprocess text by removing stopwords and tokenizing
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word.lower() for word in words if word.lower() not in stop_words]
    return filtered_words

# Train a Word2Vec model on the preprocessed texts of the topics and cluster labels
texts = [preprocess_text(text) for text in df['Topic_Name'].tolist()]
texts += [preprocess_text(text) for text in cluster_labels.values()]
w2v_model = Word2Vec(texts, min_count=1)

# Load the paraphrase-MiniLM-L6-v2 model from SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Get the embeddings for all topics and cluster labels
topic_embeddings = model.encode(df['Topic_Name'].tolist())
label_embeddings = model.encode(list(cluster_labels.values()))

# Assign topics to cluster labels based on Word Mover's Distance (WMD) similarity
wmd_instance = w2v_model.wv
wmd_instance.init_sims()
for i, topic_embedding in enumerate(topic_embeddings):
    similarities = [wmd_instance.wmdistance(preprocess_text(df.at[i, 'Topic_Name']), preprocess_text(label)) for label in cluster_labels.values()]
    closest_label_index = similarities.index(min(similarities))
    closest_label = list(cluster_labels.keys())[list(cluster_labels.values()).index(list(cluster_labels.values())[closest_label_index])]
    df.at[i, 'Cluster_Label'] = closest_label

# Print the topics under each cluster label
for label in cluster_labels.keys():
    print(f"Cluster label: {label}")
    topics = df[df['Cluster_Label'] == label]['Topic_Name'].tolist()
    for topic in topics:
        print(f"\t{topic}")


<ipython-input-23-355394af16fb>:44: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  wmd_instance.init_sims()


Cluster label: research
	Vaccination Exemption for School Children
	Maternal and Child Healthcare
	Child Vaccination
	Gaza Child Vaccination
	Democrat Criticism
	Routine Childhood Vaccination
	Prenatal mRNA Vaccine Exposure and Autism-like Behavior
	Infant Deaths Following Vaccination
	Child COVID-19 Vaccination Risks
	Vaccination of Undocumented Immigrants in NYC
	Vaccine Concerns
	Anti-Polio Vaccination in North Waziristan
	COVID-19 Vaccination of Children in Brazil
	Diphtheria Vaccination
	Vaccination of Children in Japan and the US
	Routine Infant Circumcision
	COVID-19 Vaccination and Deaths
	Diphtheria Vaccination
	Rubella Vaccination
	Childhood Vaccination
	COVID-19 Vaccination in Texas and Florida
	Mandatory Vaccination
	Vaccine Safety Research
	Rotavirus Vaccination
	Vaccine Misinformation
	Childhood Vaccination
	Child Vaccination in Gaza
Cluster label: cost
	Pertussis Vaccination
	Myocarditis side effect of COVID-19 vaccines in young men
	Vaccines and Infant Mortality
	Malari

2. Cosine similarity of TF-IDF vectors:

**Cosine similarity of TF-IDF vectors**: You can convert the topic names and cluster labels into TF-IDF vectors using the TfidfVectorizer class from the scikit-learn library, and then calculate the cosine similarity between each topic name and each cluster label. You can then assign each topic to the cluster with the highest cosine similarity.

In [9]:
# Load the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

# Load your CSV file into a pandas DataFrame
import pandas as pd
df = pd.read_csv('/content/topics_name_110.csv')

# Convert the topic names and cluster labels into TF-IDF vectors
vectorizer = TfidfVectorizer()
topic_vectors = vectorizer.fit_transform(df['Topic_Name'].tolist())
cluster_label_vectors = vectorizer.transform(list(cluster_labels.values()))

# Calculate the cosine similarity between each topic name and each cluster label
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(topic_vectors, cluster_label_vectors)

# Assign topics to cluster labels based on similarity
for i in range(len(df)):
    closest_label_index = similarities[i].argmax()
    closest_label = list(cluster_labels.keys())[list(cluster_labels.values()).index(list(cluster_labels.values())[closest_label_index])]
    df.at[i, 'Cluster_Label'] = closest_label

# Print the topics under each cluster label
for label in cluster_labels.keys():
    print(f"Cluster label: {label}")
    topics = df[df['Cluster_Label'] == label]['Topic_Name'].tolist()
    for topic in topics:
        print(f"\t{topic}")


Cluster label: research
	Vaccine Safety for Infants
	MMR Vaccination for Children
	Tetanus Vaccination
	Vaccines and Autism
	Pfizer's Failure to Disclose Risks of Preterm Birth and Neonatal Death in Pregnant Women in RSV Vaccine Trial
	Vaccination Exemption for School Children
	COVID-19 Vaccination in Pregnancy and Neonatal Outcomes
	Vaccines for Tetanus and Chickenpox
	COVID-19 Vaccination Clinic for Children
	Flu Vaccination During Pregnancy
	MMR Vaccination
	Hepatitis B Vaccination
	Maternal and Child Healthcare
	Polio Vaccination Campaign
	Pertussis Vaccination
	Child Vaccination
	Myocarditis side effect of COVID-19 vaccines in young men
	Vaccines and Infant Mortality
	Childhood Vaccination Schedule
	Malaria Vaccination in Cameroon
	Gaza Child Vaccination
	Vaccination for Rabies and Tetanus
	Child and Adolescent COVID-19 Vaccination
	Democrat Criticism
	Immunization Schedule for Babies
	Routine Childhood Vaccination
	Prenatal mRNA Vaccine Exposure and Autism-like Behavior
	Infant D

In [11]:
import gensim.downloader as api
print(api.info())

{'corpora': {'semeval-2016-2017-task3-subtaskBC': {'num_records': -1, 'record_format': 'dict', 'file_size': 6344358, 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py', 'license': 'All files released for the task are free for general research use', 'fields': {'2016-train': ['...'], '2016-dev': ['...'], '2017-test': ['...'], '2016-test': ['...']}, 'description': 'SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collected data is given in sections 3 and 4.1 of the task paper http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-report.pdf linked in section “Papers” of https://github.com/RaRe-Technologies/gensim-data/issues/18.', 'checksum': '701ea67acd82e75f95e1d8e62fb0ad29', 'file_name': 'semeval-2016-2017-task3-subtaskBC.gz',

In [24]:
# Load the SentenceTransformer model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Load your CSV file into a pandas DataFrame
import pandas as pd
df = pd.read_csv('/content/topics_name_110.csv')

# Get the embeddings for all topics and cluster labels
import torch
topic_embeddings = model.encode(df['Topic_Name'].tolist())

cluster_labels = {
    'research': 'This is a research topic',
    'cost': 'This is a cost-related topic',
    'efficacy': 'This is an efficacy-related topic',
    'epidemiology': 'This is an epidemiology topic',
    'awareness': 'This is an awareness topic',
    'preference': 'This is a preference-related topic',
    'treatment': 'This is a treatment-related topic',
    'dummy1': 'This is a dummy topic'
}
label_embeddings = model.encode(list(cluster_labels.values()))

# Assign topics to cluster labels based on similarity
from scipy.spatial.distance import cosine
for i, topic_embedding in enumerate(topic_embeddings):
    similarities = [1 - cosine(topic_embedding, label_embedding) for label_embedding in label_embeddings]
    closest_label_index = similarities.index(max(similarities))
    closest_label = list(cluster_labels.keys())[closest_label_index]
    df.at[i, 'Cluster_Label'] = closest_label

# Print the topics under each cluster label
for label in cluster_labels.keys():
    print(f"Cluster label: {label}")
    topics = df[df['Cluster_Label'] == label]['Topic_Name'].tolist()
    for topic in topics:
        print(f"\t{topic}")


Cluster label: research
Cluster label: cost
	Baby Care Logbook
	Financial Assistance for Baby's Health
	WEF Digital ID
	Pediatric Wellness Checkups
	Newsroom Exposé
Cluster label: efficacy
	Vaccine Safety for Infants
	Pfizer's Failure to Disclose Risks of Preterm Birth and Neonatal Death in Pregnant Women in RSV Vaccine Trial
	Immunization Schedule for Babies
	COVID-19 Vaccination Recommendations
	Vaccine Concerns
	COVID Vaccine Safety in Children
	COVID-19 Vaccination Recommendations
	Vaccine Safety in Children
	Vaccine Safety
	COVID-19 Vaccination Compensation
	Serum Run Nome
	Pediatric Nurse Qualifications
	Vaccine Safety in Children
	Vaccine Safety Research
	WHO guidance on clinical management of diphtheria
	Vaccine Safety Monitoring
	Vaccination Choice for Healthy Children
	Student Immunization Requirements
Cluster label: epidemiology
	Vaccination Exemption for School Children
	Vaccines for Tetanus and Chickenpox
	Hepatitis B Vaccination
	Vaccines and Infant Mortality
	Malaria Vac

The code begins by importing the necessary libraries and loading the SentenceTransformer model, which is a model for encoding sentence embeddings. The specific model used here is 'paraphrase-MiniLM-L6-v2'.

Next, the code imports the pandas library and reads the CSV file containing topic names into a pandas DataFrame called df.

The code then uses the loaded SentenceTransformer model to encode the topic names into embeddings. This is done by calling the encode method on the model with the topic names as input. The resulting embeddings are stored in the topic_embeddings variable.

The cluster labels are defined as a dictionary called cluster_labels, where the keys represent the label names and the values represent the descriptions of each label.

The code encodes the descriptions of the cluster labels using the SentenceTransformer model in a similar way as the topic embeddings. The resulting embeddings are stored in the label_embeddings variable.

The next step is to assign topics to cluster labels based on similarity. This is done using the cosine similarity measure. For each topic embedding, the code calculates the cosine similarity between the topic embedding and each label embedding. The highest similarity score determines the closest label.

The code iterates over each topic embedding and calculates the similarity scores by subtracting the cosine distances from 1. The similarities list contains the similarity scores for each label embedding.

The code identifies the index of the label embedding with the highest similarity score using the index method. This index is then used to retrieve the corresponding label key from the cluster_labels dictionary.

The closest label is assigned to the Cluster_Label column of the DataFrame for each topic using the at method.
Finally, the code prints the topics under each cluster label. It iterates over each label in the cluster_labels dictionary and filters the DataFrame to include only the topics associated with that label. The topics are then printed for each label.

Overall, this code uses the SentenceTransformer model to encode topic names and cluster label descriptions into embeddings. It then computes the cosine similarity between the topic embeddings and label embeddings to assign each topic to the closest cluster label. The topics under each cluster label are then printed for analysis or further processing.
